## setting up the base directory and initial configurations
    ### To Do
        * preliminary analysis
        * preprocessing
            * identify categorical variables
            * missing value imputation
        * Feature Selection
            * run rf with selected features
            * use GA to select features
            * encode GA urself
            

In [82]:
folder_name = "/springleaf_marketing_response/"
base_dir = "/Users/gulshan/kaggle/"
data_dir = base_dir+"/data/"+folder_name
output_dir = base_dir+"/output/"+folder_name

In [83]:
%matplotlib inline

In [84]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [85]:
train_data=pd.read_csv(data_dir+"/train.csv")
test_data=pd.read_csv(data_dir+"/test.csv")
print train_data.shape,test_data.shape

## Preliminary Analysis Pipeline
    * dtype of columns
    * missing value check
    * distribution plots

In [89]:
train_data.head()

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,...,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,1,0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,BRANCH,1


In [91]:
type(train_data.columns.values),train_data.columns.values

(numpy.ndarray,
 array(['ID', 'VAR_0001', 'VAR_0002', ..., 'VAR_1933', 'VAR_1934', 'target'], dtype=object))

In [74]:
train=train_data.ix[0:,1:-1]
target=train_data['target']

##Preprocessing Pipeline
    * Imputation of missing values
    * Encoding categorical features
    * Normalization

In [ ]:
class preprocess:
    def one_hot_encoding(input_array):
        
    

In [93]:

# Create the random forest object which will include all the parameters
# for the fit
# rf = RandomForestClassifier(n_estimators = 100,n_jobs=-1, verbose =1)
# %time rf.fit(train, target)

In [ ]:
test_data.head()

In [ ]:
output = rf.predict(test_data)